In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

In [3]:
year = [1994,1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,2018]
midterm = [1994,1998,2002,2006,2010,2014]
pres = [1996,2000,2004,2008,2012,2016]
year_val = list(range(0,7))
new_val = list(range(0,8))

### Big Picture:

Goal: Instead of having each data point have its own row, we want to join rows based on lookup number.

For example, instead of the voter registration percentage for "Asian|1994" and "Asian|1996", we want just one row that says "Race_Asian_registered" and the voting reg. data for 1996-2016


Previous:


| ID         | Voting Reg % |
|------------|--------------|
| Asian-1996 | 59.2         |
| Asian-1998 | 58.4         |

Goal:



|  - | Year           | 1996 | 1998 | 2000 | 2002 | 2004 |
|----|----------------|------|------|------|------|------|
| ID | Race_Asian_reg | 59.2 | 58.4 | 60.5 | 55.4 | 61.3 |

### Read in csv file of all data into a Pandas dataframe### 

In [4]:
df2 = pd.read_csv('big_ivr_test.csv')

In [5]:
#Store a list of all the keys. They SHOULD be unique (some of them aren't)
key = list(df2['Lookup_Item'])
key = key[:-1]

reg_list = []
voted_list = []

# For each key value, store the corresponding registration and voting percentage (the data we want)
for value in key:
    #print(value)
    temp = df2.query('Lookup_Item == @value')
    reg = list(temp['pct_reg_cvap'])
    voted = list(temp['pct_voted_cvap'])
    reg_list.append(reg[0])
    voted_list.append(voted[0])

In [19]:
# Update the dataframe to have its key be the Lookup_Item column
df3 = df2.set_index("Lookup_Item", drop = False)
print(df3.shape)

# Screen out the Lookup_Items that we don't want 
# Also removes any tag with phrase 'Not in Universe'
df3 = df3[df3.dimension != 'Not in Universe']
df3 = df3[df3.single_topline != 'Geo Area of Origin']
df3 = df3[df3.dimension != 'Ignore']
df3 = df3[df3.single_topline != 'Race, Urban/Rural Type']
df3 = df3[df3.single_topline != 'Urban/Rural Type, Income']
df3 = df3[df3.single_topline != 'Race, Urban/Rural Type, Income']
df3 = df3[df3.single_topline != 'Urban/Rural Type']
df3 = df3[df3.single_topline != 'Race, Age']
df3 = df3[df3.single_topline != 'Race, Educational Attainment']
df3 = df3[df3.dimension != 'Other']
df3 = df3[df3.single_topline != 'Race, Income']

print(df3.shape)

(9544, 12)
(1194, 12)


In [20]:
# First five rows of our new table
df3.head(13)

,rowid,single_topline,dimension,dimension1,year,cvap,reg_cvap,voted_cvap,pct_reg_cvap,pct_voted_cvap,Lookup_Item,% voted who are registered
Lookup_Item,,,,,,,,,,,,
Asian|1994,1,Race,Asian,Asian,1994,"2,683,979","1,393,527","1,056,601",51.92,39.37,Asian|1994,75.82206875
Asian|1996,2,Race,Asian,Asian,1996,"3,865,055","2,209,533","1,740,493",57.17,45.03,Asian|1996,78.77198485
Asian|1998,3,Race,Asian,Asian,1998,"4,421,769","2,163,153","1,431,391",48.92,32.37,Asian|1998,66.17150983
Asian|2000,4,Race,Asian,Asian,2000,"4,717,882","2,470,357","2,044,981",52.36,43.35,Asian|2000,82.78078836
Asian|2002,5,Race,Asian,Asian,2002,"5,310,520","2,615,263","1,660,294",49.25,31.26,Asian|2002,63.48478145
Asian|2004,6,Race,Asian,Asian,2004,"6,269,651","3,247,436","2,768,231",51.80,44.15,Asian|2004,85.2435891
Asian|2006,7,Race,Asian,Asian,2006,"6,614,289","3,244,592","2,144,606",49.05,32.42,Asian|2006,66.09786377
Asian|2008,8,Race,Asian,Asian,2008,"7,059,262","3,900,564","3,357,383",55.25,47.56,Asian|2008,86.07429592
Asian|2010,9,Race,Asian,Asian,2010,"7,638,549","3,765,413","2,353,798",49.29,30.81,Asian|2010,62.51101805


### Tricky part!!!

In [14]:
# This is a bunch of annoying, confusing code.

'''
Basically, the original data has a general rule that each chunk of
inputs is separated by a column with the year 2016.
After the year 2016 appears, a Lookup_Item is being queried.

This code goes through each row in the table and grabs 
the first Lookup_Item and the last Lookup_Item for each sequence.

For example, it will put 'Asian|1994' as the first entry in the list
'start' and it will put 'Asian|2016' as the first entry in the list 'end'

Thus, when one prints the first entry of list start and list end, the output
will be: Asian|1994, Asian|2016. 

This will allow me to retrieve chunks of data from the original table.
''' 

num_rows = df3.shape[0]

start = []
end = []

first = True
for i in range(0,num_rows):
    year = str(int(df3.iloc[i,4]))
    dim = str(df3.iloc[i,2])
    combo = dim + '|' + year
    
    #Special case for first year
    if first == True:
        start.append(combo)
        first = False
    
    if year == '2016':
        end.append(combo)
        
        new_year = str(int(df3.iloc[i+1,4]))
        new_dim = str(df3.iloc[i+1,2])
        new_start = new_dim + '|' + new_year
        start.append(new_start)
start = start[:-1]

In [15]:
# Double check to ensure that the sizes of each list are the same
print(len(start),len(end))

# for a,b in zip(start,end):
#     print(a,b)

print(start[10],end[10])

180 180
18 - 24|1994 18 - 24|2016


### Fun part! 


In [16]:
id_list = []
data_list = []

'''
For each pair of Lookup_Items from the previous step,
grab the registration percentage and voting percentage data.

Insert both of them into a new table, with a Unique Identifier
based on the info from the table. 

For example, Asian|1994 . . . Asian|2016 becomes Race_Asian_r (for registered)
and Race_Asian_v (for voted)
'''
for a,b in zip(start,end):
    cur = df3.loc[a:b,"pct_reg_cvap":"pct_voted_cvap"]
    head = df3.loc[a,"single_topline"]
    dim = df3.loc[a,"dimension"]
    
    reg = cur.values[:,0]
    vote = cur.values[:,1]
    
    while(len(reg) < 12):
        reg = np.insert(reg,0, 0)
    
    while(len(vote) < 12):
        vote = np.insert(vote,0,0)
    
    #Unique_ID code
    reg_id = str(head) + '_' + str(dim) + '_r'
    vote_id = str(head) + '_' + str(dim) + '_v'


    id_list.append(reg_id)
    id_list.append(vote_id)
    data_list.append(reg)
    data_list.append(vote)
    

In [21]:
print(id_list)

['Race_Asian_r', 'Race_Asian_v', 'Race_Black_r', 'Race_Black_v', 'Race_Combination_r', 'Race_Combination_v', 'Race_Hawaiian/Pacific Islander_r', 'Race_Hawaiian/Pacific Islander_v', 'Race_Native American_r', 'Race_Native American_v', 'Race_White_r', 'Race_White_v', 'Race_Hispanic_r', 'Race_Hispanic_v', 'Race_Non-Hispanic_r', 'Race_Non-Hispanic_v', 'Race_Non-White or Hispanic_r', 'Race_Non-White or Hispanic_v', 'Race_White Non-Hispanic_r', 'Race_White Non-Hispanic_v', 'Age_18 - 24_r', 'Age_18 - 24_v', 'Age_25 - 29_r', 'Age_25 - 29_v', 'Age_30 - 39_r', 'Age_30 - 39_v', 'Age_40 - 49_r', 'Age_40 - 49_v', 'Age_50 - 65_r', 'Age_50 - 65_v', 'Age_65+_r', 'Age_65+_v', 'Income_0K_12.5K_r', 'Income_0K_12.5K_v', 'Income_100K+_r', 'Income_100K+_v', 'Income_12.5K_20K_r', 'Income_12.5K_20K_v', 'Income_20K_40K_r', 'Income_20K_40K_v', 'Income_40K_60K_r', 'Income_40K_60K_v', 'Income_60K_75K_r', 'Income_60K_75K_v', 'Income_75K_100K_r', 'Income_75K_100K_v', 'Gender and Marriage_Married Men_r', 'Gender and 

In [17]:
# Put the info together and export to file!
final = np.column_stack([id_list,data_list])
print(final[0])

with open("ivr_final.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(final)

['Race_Asian_r'
 array([51.92, 57.17, 48.92, 52.36, 49.25, 51.8 , 49.05, 55.25, 49.29,
       56.32, 48.84, 56.25])]


In [18]:
yuh = pd.read_csv('ivr_final.csv')
yuh.head(220)

,Race_Asian_r,[51.92 57.17 48.92 52.36 49.25 51.8 49.05 55.25 49.29 56.32 48.84 56.25]
0,Race_Asian_v,[39.37 45.03 32.37 43.35 31.26 44.15 32.42 47....
1,Race_Black_r,[61.27 66.4 63.56 67.46 62.34 68.68 60.94 69....
2,Race_Black_v,[38.87 52.99 41.78 56.77 42.18 60.04 41.01 64....
3,Race_Combination_r,[ 0. 0. 0. 0. 0. 69.48 63.1 68....
4,Race_Combination_v,[ 0. 0. 0. 0. 0. 58.25 39.9 60....
5,Race_Hawaiian/Pacific Islander_r,[ 0. 0. 0. 0. 0. 58.82 46.74 57....
6,Race_Hawaiian/Pacific Islander_v,[ 0. 0. 0. 0. 0. 49.16 29.42 50....
7,Race_Native American_r,[56.35 60.56 58.34 59.7 53.18 57.04 54.17 60....
8,Race_Native American_v,[37.76 44.68 36.13 47.71 29.58 45.58 30.13 47....
9,Race_White_r,[68.42 72.04 68.21 70.43 67.87 73.59 69.51 72....
